# 3 实验：波士顿房屋价格预测

本节使用线性回归的python实现方法，应用案例数据集，来详细说明线性回归模型是如何应用的。

<center><img src='image/boston_houses.jpeg' width=400></center>



数据集是1970年波士顿各地区的房价和其他统计信息，包含506个样本，每个样本包含14个属性，如下表所示：

|属性|翻译|定义|
|---|---|---|
|CRIM|犯罪率|per capita crime rate by town|
|ZN|住宅用地比例|proportion of residential land zoned for lots over 25,000 sq.ft.|
|INDUS|非零售商业用地比例|proportion of non-retail business acres per town.|
|CHAS|是否临河，1为是，0为否|Charles River dummy variable (1 if tract bounds river; 0 otherwise)|
|NOX|氮氧化物浓度|nitric oxides concentration (parts per 10 million)|
|RM|每栋住宅平均房间数|average number of rooms per dwelling|
|AGE|1940年以前建筑比例|proportion of owner-occupied units built prior to 1940|
|DIS|与市中心的距离|weighted distances to five Boston employment centres|
|RAD|高速公路的可接近性|index of accessibility to radial highways|
|TAX|房地产税率|full-value property-tax rate per $10,000 |
|PTRATIO| 师生比例|pupil-teacher ratio by town|
|B|有色人种比例|1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town|
|LSTAT|低端人口比例| lower status of the population|
|MEDV|房价中位数|Median value of owner-occupied homes in \$1000's|





该数据集用于回归问题，包含13个自变量，因变量为**MEDV 房价中位数**。  
数据集来源：  
The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.  


## 1. 读取数据

### 实验3.1 读取数据
使用pandas库来读取路径下的数据："datasets/波士顿房价数据.csv"

In [1]:
EN_CN_dict = {'CRIM': '犯罪率',
             'ZN': '住宅用地比例',
             'INDUS': '非零售商业用地比例',
             'CHAS': '是否临河',
             'NOX': '氮氧化物浓度',
             'RM': '每栋住宅平均房间数',
             'AGE': '1940年以前建筑比例',
             'DIS': '与市中心的距离',
             'RAD': '高速公路的可接近性',
             'TAX': '房地产税率',
             'PTRATIO': '师生比例',
             'B': '有色人种比例',
             'LSTAT': '低端人口比例',
             'MEDV': '房价中位数'}

### 实验3.2 描述性统计分析
应用pandas的描述统计方法，了解该数据集的基本统计特性

## 2. 数据预处理
### 实验3.3 处理重复值
首先，检查下数据是否存在重复，使用`DataFrame.duplicated()`检查是否存在重复的行，`True`代表为重复行，`False`代表不是重复行。

### 实验3.4 处理缺失值
其次，检查是否存在缺失值问题。使用`DataFrame.isnull()`返回`True`代表有，`False`代表没有。

### 实验3.5 处理异常值
检查是否存在异常值问题。

### 实验3.6 标准化处理
常用的标准化处理方法有最小最大值标准化处理、均值标准差标准化处理、小数标度标准化处理等。下面我们采用均值标准差标准化处理：

$$
standardize(S) = \frac{S - \mu_S}{\sigma_S}
$$

### 2.5 生成解释变量（自变量）X和被解释变量（因变量）y

生成模型的解释变量x和被解释变量y

In [16]:
X = df[['犯罪率', '住宅用地比例', '非零售商业用地比例', '氮氧化物浓度', '每栋住宅平均房间数', '1940年以前建筑比例',
       '与市中心的距离', '高速公路的可接近性', '房地产税率', '师生比例', '有色人种比例', '低端人口比例' ]]
y = df['房价中位数']

## 3. 相关性分析
### 实验3.7 计算相关系数矩阵

在做多元回归之前，我们需要确定因变量（被解释变量）和自变量（解释变量）之间是否存在相关性。可以使用`pandas.dataframe.corr()`来计算相关系数矩阵：

## 4. 样本内和样本外划分

随机对照试验是商业数据分析领域中常用的一种实验设计方法。在随机对照试验中，研究者将参与试验的个体随机分为两组：实验组和对照组。实验组接受某种干预或处理，而对照组则不接受干预，作为对照参照。

根据上述的理论，我们通常将数据集划分为训练集（training sets）和测试集（test sets），前者负责模型的训练任务，后者用来评价模型的表现。两者的划分比例按照经验，设置为0.8:0.2。

一般将80%的原始数据集的子样本集作为样本内(In-sample, IS)，剩余20%作为样本外(Out-of-sample, OOS)。样本总数为500个，一种简单的切割方法是直接指定前80%个样本为IS，后面20%为OOS，但是这种采样方法存在弊端。

![fix_split](image/fix_split.png)

当我们并不了解原始数据集是不是被刻意排列了，最好的办法是使用随机抽样，即随机抽80%为IS，剩余的20%为OOS。我们使用生成随机数的方法，生成随机序列，来从总体样本中随机抽取样本：

![random_split](image/random_split.png)

In [20]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y,train_size=0.8,random_state=42)

## 5. 多元回归分析

### 实验3.8 样本内训练

模型的回归方程是

$$
房价中位数 = (？)\times犯罪率+(？)\times住宅用地比例+(？)\times非零售商业用地比例\\+(？)\times氮氧化物浓度+(？)\times每栋住宅平均房间数\\+(？)\times1940年以前建筑比例+(？)\times与市中心的距离+(？)\times高速公路的可接近性+(？)\times房地产税率\\+(？)\times师生比例+(？)\times有色人种比例+(？)\times低端人口比例+（？）
$$

### 实验3.9 样本外表现

我们再来看看我们刚才训练的模型，在样本外的表现。

## 6. 结论

最后，我们根据回归方程：

$$
房价中位数 = (？)\times犯罪率+(？)\times住宅用地比例+(？)\times非零售商业用地比例\\+(？)\times氮氧化物浓度+(？)\times每栋住宅平均房间数\\+(？)\times1940年以前建筑比例+(？)\times与市中心的距离+(？)\times高速公路的可接近性+(？)\times房地产税率\\+(？)\times师生比例+(？)\times有色人种比例+(？)\times低端人口比例+（？）
$$

可以得出以下结论，在保持其他自变量不变的情况下：
1. 每增加一个单位的**每栋住宅平均房间数**，都会带来？个单位的**房价中位数**上涨; 
2. 每增加一个单位的**与市中心的距离**，都会带来？个单位的**房价中位数**下降; 
3. 每增加一个单位的**都低端人口比例**，会带来？个单位的**房价中位数**下降; 
4. 每增加一个单位的**高速公路的可接近性**, 都会带来？个单位的**房价中位数**上涨; 


## 7. 思考

在结束此案例分析之前，我们思考以下问题：
1. 从1978年开始收集的数据与今天的数据相似性如何、是否还有参照意义？通货膨胀是否要考虑?
2. 数据中的特征是否足以描述房屋价格？你认为装修质量、公摊面积、是否有花园等因素应该考虑进去吗?
3. 在城市收集的数据适用于农村和集镇吗?
4. 如果一个房屋所有者想要出售他的房屋，他应用此模型预测了他的房屋价格，你认为他该如何说服买家相信这个模型给出的价格？